In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for entry in os.listdir('/kaggle/input/birdclef-2024/train_audio'):
print(os.listdir('/kaggle/input/birdclef-2024/train_audio'))
print(len(os.listdir('/kaggle/input/birdclef-2024/train_audio')))

train_species = os.listdir('/kaggle/input/birdclef-2024/train_audio')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


['redspu1', 'zitcis1', 'marsan', 'rewlap1', 'grenig1', 'insbab1', 'commyn', 'labcro1', 'yebbul3', 'whtkin2', 'brasta1', 'brnhao1', 'bkcbul1', 'grywag', 'graher1', 'brodro1', 'purher1', 'bkskit1', 'eurbla2', 'grtdro1', 'comtai1', 'paisto1', 'brcful1', 'grehor1', 'whcbar1', 'blakit1', 'integr', 'gryfra', 'grnsan', 'whbsho3', 'sqtbul1', 'greegr', 'wemhar1', 'comsan', 'rewbul', 'cohcuc1', 'heswoo1', 'gybpri1', 'gargan', 'darter2', 'rerswa1', 'grnwar1', 'isbduc1', 'brwjac1', 'mawthr1', 'eaywag1', 'pursun3', 'lblwar1', 'kerlau2', 'junmyn1', 'rutfly6', 'oripip1', 'houspa', 'lobsun2', 'wynlau1', 'eucdov', 'compea', 'spodov', 'nutman', 'revbul', 'aspswi1', 'blaeag1', 'whiter2', 'litspi1', 'litswi1', 'rufwoo2', 'rossta2', 'barfly1', 'ruftre2', 'inpher1', 'maghor2', 'litgre1', 'grynig2', 'lirplo', 'rorpar', 'gloibi', 'comkin1', 'orihob2', 'sohmyn1', 'yebbab1', 'plapri1', 'stbkin1', 'blhori1', 'comros', 'ashdro1', 'dafbab1', 'ingori1', 'rocpig', 'jerbus2', 'inbrob1', 'houcro1', 'asbfly', 'whbwat1'

Notes about intial data investigation
1. for taxonomy, species_group is going to be not always populated. the dataset seems to only populate the same entry for the group. Need to backfill all species group. Reported as are all NaN for the 182 species we concern. So probably don't need to handle it now.
2. 

In [2]:
taxonomy = pd.read_csv('/kaggle/input/birdclef-2024/eBird_Taxonomy_v2021.csv')
taxonomy.head()


,TAXON_ORDER,CATEGORY,SPECIES_CODE,PRIMARY_COM_NAME,SCI_NAME,ORDER1,FAMILY,SPECIES_GROUP,REPORT_AS
0,1,species,ostric2,Common Ostrich,Struthio camelus,Struthioniformes,Struthionidae (Ostriches),Ostriches,NaN
1,6,species,ostric3,Somali Ostrich,Struthio molybdophanes,Struthioniformes,Struthionidae (Ostriches),NaN,NaN
2,7,slash,y00934,Common/Somali Ostrich,Struthio camelus/molybdophanes,Struthioniformes,Struthionidae (Ostriches),NaN,NaN
3,8,species,grerhe1,Greater Rhea,Rhea americana,Rheiformes,Rheidae (Rheas),Rheas,NaN
4,14,species,lesrhe2,Lesser Rhea,Rhea pennata,Rheiformes,Rheidae (Rheas),NaN,NaN


In [3]:
train_taxonomy = taxonomy[taxonomy['SPECIES_CODE'].isin(train_species)]
species_group = taxonomy[taxonomy['REPORT_AS'].notna()]
species_group.head()

,TAXON_ORDER,CATEGORY,SPECIES_CODE,PRIMARY_COM_NAME,SCI_NAME,ORDER1,FAMILY,SPECIES_GROUP,REPORT_AS
5,15,issf,lesrhe4,Lesser Rhea (Puna),Rhea pennata tarapacensis/garleppi,Rheiformes,Rheidae (Rheas),NaN,lesrhe2
6,18,issf,lesrhe3,Lesser Rhea (Darwin's),Rhea pennata pennata,Rheiformes,Rheidae (Rheas),NaN,lesrhe2
9,21,issf,higtin2,Highland Tinamou (South American),Nothocercus bonapartei [bonapartei Group],Tinamiformes,Tinamidae (Tinamous),NaN,higtin1
10,26,issf,higtin3,Highland Tinamou (Costa Rican),Nothocercus bonapartei frantzii,Tinamiformes,Tinamidae (Tinamous),NaN,higtin1
23,73,issf,brntin1,Brown Tinamou (castaneus),Crypturellus obsoletus castaneus,Tinamiformes,Tinamidae (Tinamous),NaN,brotin1
